**Set environment**

In [1]:
suppressMessages(suppressWarnings(source("../run_config_project_sing.R")))
show_env()

You are working on        Singularity: singularity_proj_encode_fcc 
BASE DIRECTORY (FD_BASE): /data/reddylab/Kuei 
REPO DIRECTORY (FD_REPO): /data/reddylab/Kuei/repo 
WORK DIRECTORY (FD_WORK): /data/reddylab/Kuei/work 
DATA DIRECTORY (FD_DATA): /data/reddylab/Kuei/data 

You are working with      ENCODE FCC 
PATH OF PROJECT (FD_PRJ): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC 
PROJECT RESULTS (FD_RES): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/results 
PROJECT SCRIPTS (FD_EXE): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/scripts 
PROJECT DATA    (FD_DAT): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/data 
PROJECT NOTE    (FD_NBK): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/notebooks 
PROJECT DOCS    (FD_DOC): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/docs 
PROJECT LOG     (FD_LOG): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/log 
PROJECT REF     (FD_REF): /data/reddylab/Kuei/repo/Proj_ENCODE_FCC/references 



## Import data

In [2]:
txt_fdiry = file.path(
    FD_RES, 
    "region_annotation", 
    "fcc_astarr_macs_input_overlap",
    "summary"
)
vec = dir(txt_fdiry)
for (txt in vec){cat(txt, "\n")}

matrix.annotation.encode_chipseq_flagship.tsv 
matrix.annotation.encode_chipseq_full.tsv 
matrix.annotation.encode_chipseq_histone.tsv 
matrix.annotation.encode_chipseq_subset.tsv 
matrix.annotation.fcc_peak_call.tsv 
matrix.annotation.genome_tss.tsv 
matrix.annotation.merge.fcc.tss.chipseq.tsv 
region.annotation.encode_chipseq_flagship.tmp.tsv 
region.annotation.encode_chipseq_flagship.tsv 
region.annotation.encode_chipseq_histone.tsv 
region.annotation.encode_chipseq_subset.tsv 
region.annotation.encode_chromatin_states.tsv 
region.annotation.encode_e2g_benchmark.tsv 
region.annotation.fcc_astarr_csaw.tsv 
region.annotation.fcc_crispri_growth.tsv 
region.annotation.fcc_crispri_hcrff.tsv 
region.annotation.fcc_mpra_agarwal2023.tsv 
region.annotation.fcc_starrmpra_junke.tsv 
region.annotation.genome_tss.tsv 
region.annotation.module_tf_shannon.tsv 
region.intersect.encode_chipseq_flagship.tmp.tsv 
region.intersect.encode_chipseq_flagship.tsv 
region.intersect.encode_chipseq_histone.tsv

In [3]:
txt_fdiry = file.path(
    FD_RES, 
    "region_annotation", 
    "fcc_astarr_macs_input_overlap",
    "summary"
)
txt_fname = "region.annotation.encode_chromatin_states.tsv"
txt_fpath = file.path(txt_fdiry, txt_fname)

dat = read_tsv(txt_fpath, show_col_types = FALSE)

dat_region_annot_import = dat
print(dim(dat))
fun_display_table(head(dat, 3))

[1] 418821     10


Chrom,ChromStart,ChromEnd,Region,Annotation_A,Annotation_B,Group,Label,Region_Annot,Region_Count
chr1,10038,10405,chr1:10038-10405,fcc_astarr_macs_input_overlap,encode_ccres_ENCFF286VQG,cCREs,Low-DNase,chr1:10033-10250;chr1:10385-10713,2
chr1,10038,10405,chr1:10038-10405,fcc_astarr_macs_input_overlap,encode_chromhmm_ENCFF106BGJ,ChromHMM,Quies,chr1:0-16000,1
chr1,14282,14614,chr1:14282-14614,fcc_astarr_macs_input_overlap,encode_chromhmm_ENCFF106BGJ,ChromHMM,Quies,chr1:0-16000,1


In [5]:
dat = dat_region_annot_import
res = table(dat$Group)
print(res)


               cCREs cCREs:REST+ silencer cCREs:STARR silencer 
              193629                 3843                  383 
            ChromHMM 
              220966 


In [24]:
dat = dat_region_annot_import
lst = list(
    "encode_chromatin_states.full"     = dat %>% dplyr::filter(Group %in% c("cCREs", "ChromHMM")),
    "encode_chromatin_states.ccres"    = dat %>% dplyr::filter(Group == "cCREs"),
    "encode_chromatin_states.chromhmm" = dat %>% dplyr::filter(Group == "ChromHMM")
)
lst = lapply(lst, function(dat){
    dat = dat %>% dplyr::select(Chrom:Region, Group, Label) %>% dplyr::distinct()
    return(dat)
})

lst_dat = lst

In [25]:
lst = lst_dat
head(lst[[1]], 3)

Chrom,ChromStart,ChromEnd,Region,Group,Label
<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>
chr1,10038,10405,chr1:10038-10405,cCREs,Low-DNase
chr1,10038,10405,chr1:10038-10405,ChromHMM,Quies
chr1,14282,14614,chr1:14282-14614,ChromHMM,Quies


In [26]:
lst = lst_dat
lst = lapply(lst, function(dat){
    dat = dat %>% 
        dplyr::select(-Group) %>%
        dplyr::mutate(Value = 1) %>% 
        tidyr::spread(Label, Value) %>%
        replace(is.na(.), 0)
    return(dat)
})
lst_mat = lst

In [27]:
lst = lst_mat
head(lst[[1]], 3)

Chrom,ChromStart,ChromEnd,Region,Biv,CA-CTCF,CA-H3K4me3,CA-only,CA-TF,dELS,⋯,PLS,Quies,ReprPC,Tss,TssFlnk,TssFlnkD,TssFlnkU,Tx,TxWk,ZNF/Rpts
<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,10038,10405,chr1:10038-10405,0,0,0,0,0,0,⋯,0,1,0,0,0,0,0,0,0,0
chr1,14282,14614,chr1:14282-14614,0,0,0,0,0,0,⋯,0,1,0,0,0,0,0,0,0,0
chr1,16025,16338,chr1:16025-16338,0,0,0,0,0,0,⋯,0,1,0,0,0,0,0,0,1,0


In [30]:
txt_fdiry = file.path(
    FD_RES, 
    "region_annotation", 
    "fcc_astarr_macs_input_overlap",
    "summary"
)

lst = lst_dat
for (idx in names(lst)){
    dat = lst[[idx]]
    cat(idx, "\n")
    
    txt_fname = paste("region", "summary", idx, "tsv", sep = ".")
    txt_fpath = file.path(txt_fdiry, txt_fname)
    cat(txt_fname, "\n")

    write_tsv(dat, txt_fpath)
}

encode_chromatin_states.full 
region.summary.encode_chromatin_states.full.tsv 
encode_chromatin_states.ccres 
region.summary.encode_chromatin_states.ccres.tsv 
encode_chromatin_states.chromhmm 
region.summary.encode_chromatin_states.chromhmm.tsv 


In [31]:
txt_fdiry = file.path(
    FD_RES, 
    "region_annotation", 
    "fcc_astarr_macs_input_overlap",
    "summary"
)

lst = lst_mat
for (idx in names(lst)){
    dat = lst[[idx]]
    cat(idx, "\n")
    
    txt_fname = paste("matrix", "annotation", idx, "tsv", sep = ".")
    txt_fpath = file.path(txt_fdiry, txt_fname)
    cat(txt_fname, "\n")

    write_tsv(dat, txt_fpath)
}

encode_chromatin_states.full 
matrix.annotation.encode_chromatin_states.full.tsv 
encode_chromatin_states.ccres 
matrix.annotation.encode_chromatin_states.ccres.tsv 
encode_chromatin_states.chromhmm 
matrix.annotation.encode_chromatin_states.chromhmm.tsv 
